In [1]:
import os 
import imp
import warnings
import numpy as np
import pandas as pd
import importlib as imp
import itertools
from tqdm import tqdm, tqdm_notebook
from utils import tools
warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

PATH_TO_BENCHMARKS = './/data//'

**Задача**

*Ваша четвертая лабораторная работа будет заключаться в реализации одного из двух алгоритмов: <br>
метод имитации обжига или генетический алгоритм со встроенной эвристикой в виде локального поиска.  <br>
Алгоритмы реализуются для задачи Cell formation problem.* <br>

Реализовать можно на любом языке. <br>
Также нужно будет сравнить время работы алгоритма, а так же показать лучшее решение алгоритма, которое удалось найти вам. <br>

**Данные:**

Данные для работы прикрепил архивом к письму. **Структура такая:** <br>

m p (number of machines and parts) <br>
Next m rows: <br>
m(row number) list of parts processed by machine m separated by space <br>
e.g: <br>
1 9 17 19 31 33
means machine 1 processes parts 9 17 19 31 33 <br>

**Выходной формат для ответов:** <br>
Output file: instancename.sol (e.g. 20x20.sol) <br>
Output file format: <br>
m1_clusterId m2_clusterId ... - machines to clusters mapping <br>
p1_clusterId p2_clusterId ... - parts to clusters mapping <br>

In [2]:
txtfiles = [f for f in os.listdir(PATH_TO_BENCHMARKS) if os.path.isfile(os.path.join(PATH_TO_BENCHMARKS, f))]
txtfiles

['20x20.txt', '24x40.txt', '30x50.txt', '30x90.txt', '37x53.txt']

In [3]:
tools = imp.reload(tools)
machine_part_matrix = tools.get_data(PATH_TO_BENCHMARKS + txtfiles[0])
machine_part_matrix_sample = np.array([[1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

In [4]:
pd.DataFrame(machine_part_matrix_sample)

,0,1,2,3,4
0,1,0,0,1,0
1,0,1,1,0,1
2,1,0,0,0,0
3,0,1,1,0,0
4,0,0,0,1,0


Построение $similarity\ matrix$ по формуле $$S_{ij} = \frac{a_{ij}}{a_{ij} + b_{ij} + c_{ij}}$$
$a_{ij}$ - кол-во машин, которые обрабатывают детали $i, j$
$b_{ij}$ - кол-во машин, которые обрабатывают деталь $i$, но не $j$
$c_{ij}$ - кол-во машин, которые обрабатывают деталь $j$, но не $i$

In [6]:
pd.DataFrame(similarity_matrix)

,0,1,2,3,4
0,-1.0,0.0,0.0,0.333333,0.0
1,-1.0,-1.0,1.0,0.000000,0.5
2,-1.0,-1.0,-1.0,0.000000,0.5
3,-1.0,-1.0,-1.0,-1.000000,0.0
4,-1.0,-1.0,-1.0,-1.000000,-1.0


Матрицу удобнее представить в виде списка: $[[(i, j), S_{i, j}]]$

In [7]:
np_similarity

[[(1, 2), 1.0], [(1, 4), 0.5], [(2, 4), 0.5], [(0, 3), 0.3333333333333333]]

Итерируясь в порядке убывания по списку составленному по $similarity\ matrix$ генерируем первоначальное решение (разбиение на $cells$)

In [25]:
cells_p

{0: [5, 8, 15],
 1: [4, 14],
 2: [2, 7, 19, 0, 6],
 3: [10, 12],
 4: [13, 17, 3, 18],
 5: [1, 16, 9, 11]}

$Cell 1:$ [1, 2, 4]
$Cell 2:$ [0, 3]

In [9]:
pd.DataFrame(machine_part_matrix_sample)[[1, 2, 4, 0, 3]]

,1,2,4,0,3
0,0,0,0,1,1
1,1,1,1,0,0
2,0,0,0,1,0
3,1,1,0,0,0
4,0,0,0,0,1


In [134]:
tools = imp.reload(tools)
machine_part_matrix = tools.get_data(PATH_TO_BENCHMARKS + txtfiles[0])
cells_p, cells_m = tools.get_solution(machine_part_matrix, 8)

In [150]:
for k in list(cells_m.values()):
    print(k)

[0, 3, 5, 6, 8]
[4, 14, 15, 16]
[1, 17]
[2, 9, 12]
[7, 10, 11, 13, 18]
[19]


In [132]:
df = tools.solution_df(cells_p, cells_m, machine_part_matrix)
tools.compute_grouping_efficacy(cells_p, cells_m, df)

0.34558823529411764

In [146]:
from algorithm import SimulatedAnnealing

SimulatedAnnealing = imp.reload(SimulatedAnnealing)
params = {
    'initial_temperature' : 1,
    'final_temperature'   : 2,
    'chain_len'           : 3,
    'len_of_period'       : 4,
    'numb_of_cells'       : 5,
}

SA = SimulatedAnnealing.SimulatedAnnealing(machine_part_matrix)
SA.set_params(params)
SA.solve()

solution       : {'current': {'p': {0: [5, 8, 15], 1: [4, 14, 1, 16, 9, 11], 2: [2, 7, 19, 0, 6], 3: [10, 12], 4: [13, 17, 3, 18]}, 'm': {0: [0, 3, 5, 6, 8, 14], 1: [1, 16], 2: [17, 19], 3: [2, 4, 9, 12, 15], 4: [7, 10, 11, 13, 18]}}, 'neigh': {'p': {0: [5, 8, 15], 1: [4, 14, 1, 16, 9, 11], 2: [2, 7, 19, 0, 6], 3: [10, 12], 4: [13, 17, 3, 18]}, 'm': {0: [0, 3, 5, 6, 8, 14], 1: [1, 16], 2: [17, 19], 3: [2, 4, 9, 12, 15], 4: [7, 10, 11, 13, 18]}}, 'best': {'p': {0: [5, 8, 15], 1: [4, 14, 1, 16, 9, 11], 2: [2, 7, 19, 0, 6], 3: [10, 12], 4: [13, 17, 3, 18]}, 'm': {0: [0, 3, 5, 6, 8, 14], 1: [1, 16], 2: [17, 19], 3: [2, 4, 9, 12, 15], 4: [7, 10, 11, 13, 18]}}, 'initial': {'p': {0: [5, 8, 15], 1: [4, 14, 1, 16, 9, 11], 2: [2, 7, 19, 0, 6], 3: [10, 12], 4: [13, 17, 3, 18]}, 'm': {0: [0, 3, 5, 6, 8, 14], 1: [1, 16], 2: [17, 19], 3: [2, 4, 9, 12, 15], 4: [7, 10, 11, 13, 18]}}}
temperature    : {'initial': 1, 'final': 2}
cooling_rate   : 0
L: 3,        D : 4
number of cells: {'initial': 5, 'opti